# Payment Calendars Example
In this notebook we will create a portfolio containing four bonds, each one having a different roll convention. We will then run a valuation on the portfolio and retrieve the payment dates for each bond, which will each have a different payment date.

The calendars used in this example come from our partner, CoppClark, to use these in your LUSID domain, you will require the coresponding license. Please contact Finbourne to request the correct license. 

For more information on this please refer to the following Knowedge Base page: https://support.lusid.com/knowledgebase/article/KA-01995/en-us

The calendar date we are using for this example is the 4th July. As this is a market holiday in the US every year, we are creating these as US bonds. 

The following roll conventions are used:
- NoAdjustment: This will cause the bonds' payment date to be the 4th July.
- Following: This will cause the bonds' payment date to be the 5th July.
- ModifiedFollowing: This will cause the bonds' payment date to be the 5th July.
- Previous: This will cause the bonds' payment date to be the 3rd July.

## Contents
- [Setup](#setup)
- [1. Create a Portfolio](#1-create-a-portfolio)
- [2. Create and Upsert Bonds into Lusid](#2-create-and-upsert-bonds-into-lusid)
- [3. Create and Upsert Transactions](#3-create-and-upsert-transactions)
- [4. Add Quotes for Bonds](#4-add-quotes-for-all-defined-bonds)
- [5. Create and Run a Valuation](#5-create-and-run-a-valuation-on-the-portfolio)

## Setup

In [1]:
# Import generic non-LUSID packages
import os
import json
import pytz
import pandas as pd
from datetime import datetime

# Import key modules from the LUSID package
import lusid as lu
import lusid.models as lm

# Import key functions from Lusid-Python-Tools and other packages
from lusidtools.lpt.lpt import to_date
from lusidjam import RefreshingToken


# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = lu.utilities.ApiClientFactory(
    token=RefreshingToken(), api_secrets_filename=secrets_path
)

print("LUSID Environment Initialised")
print(
    "LUSID API Version :",
    api_factory.build(lu.api.ApplicationMetadataApi).get_lusid_versions().build_version,
)

LUSID Environment Initialised
LUSID API Version : 0.6.10858.0


In [2]:
# LUSID api definitions
portfolio_api = api_factory.build(lu.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lu.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.api.InstrumentsApi)
quotes_api = api_factory.build(lu.api.QuotesApi)
configuration_recipe_api = api_factory.build(lu.api.ConfigurationRecipeApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
aggregration_api = api_factory.build(lu.api.AggregationApi)

In [3]:
# Define scope and portfolio code
scope = "payment-calendars-example"
portfolio_code = "CalendarsExample"

## 1. Create a portfolio

This will be the portfolio used throughout this example.

In [4]:
try:
    transaction_portfolios_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency="USD",
            created="2010-01-01",
            sub_holding_keys=[],
        ),
    )
except lu.ApiException as e:
    print(json.loads(e.body)["title"])

Could not create a portfolio with id 'CalendarsExample' because it already exists in scope 'payment-calendars-example'.


## 2. Create and upsert bonds into LUSID

We will now create a function for upserting a customisable bond instrument.

### Create bond definition

In [5]:
def create_bond(
    currency,
    payment_frequency,
    roll_convention,
    day_count_convention,
    payment_calendars,
    reset_calendars,
    settle_days,
    reset_days,
    start_date,
    maturity_date,
    dom_ccy,
    principal,
    coupon_rate,
    bond_identifier,
    bond_name,
):

    flow_conventions = lm.FlowConventions(
        currency=currency,
        payment_frequency=payment_frequency,
        roll_convention=roll_convention,
        day_count_convention=day_count_convention,
        payment_calendars=payment_calendars,
        reset_calendars=reset_calendars,
        settle_days=settle_days,
        reset_days=reset_days,
    )

    bond = lm.Bond(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=dom_ccy,
        principal=principal,
        coupon_rate=coupon_rate,
        flow_conventions=flow_conventions,
        identifiers={},
        instrument_type="Bond",
        calculation_type="Standard",
    )

    # define the instrument to be upserted
    bond_definition = lm.InstrumentDefinition(
        name=bond_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(bond_identifier)},
        definition=bond,
    )

    # upsert the instrument
    upsert_request = {bond_identifier: bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    bond_luid = upsert_response.values[bond_identifier].lusid_instrument_id
    print(bond_luid)

### Upserting Bonds

Here we will define various bonds with different roll conventions.

In [6]:
# Define various roll conventions and associate with a bond identifier
bond_roll_mapping = {
    "A_3_02-15-2049": "Previous",
    "B_3_02-15-2049": "NoAdjustment",
    "C_3_02-15-2049": "ModifiedFollowing",
    "D_3_02-15-2049": "Following"
}

roll_df = pd.Series(bond_roll_mapping, name='roll')
roll_df.index.name = 'id'

In [7]:
# Upsert four bonds corresponding to the mapping above
for identifier, roll_convention in bond_roll_mapping.items():
    currency = "USD"
    payment_frequency = "1M"
    roll_convention = roll_convention
    day_count_convention = "ActualActual"
    payment_calendars = ["USD"]
    reset_calendars = ["USD"]
    settle_days = 0
    reset_days = 0
    start_date = datetime(2019, 3, 4, 0, tzinfo=pytz.utc)
    maturity_date = datetime(2049, 3, 4, 0, tzinfo=pytz.utc)
    dom_ccy = "USD"
    principal = 1
    coupon_rate = 0.03
    bond_identifier = identifier
    bond_name = identifier.replace("_", " ")

    create_bond(
        currency,
        payment_frequency,
        roll_convention,
        day_count_convention,
        payment_calendars,
        reset_calendars,
        settle_days,
        reset_days,
        start_date,
        maturity_date,
        dom_ccy,
        principal,
        coupon_rate,
        bond_identifier,
        bond_name,
    )

LUID_00003D5M
LUID_00003D5N
LUID_00003D5O
LUID_00003D5P


## 3. Create and upsert transactions

In order for valuations to be executed, transactions relating to the purchasing of the bonds needs to be added to the previously created portfolio.

In [8]:
transactions = pd.read_csv("data/bond_transactions.csv")
transactions

,txn_id,type,client_id,trade_date,settlement_date,quantity,price,total_consideration,currency
0,txn001,StockIn,A_3_02-15-2049,2019-03-31T10:00:00Z,2019-04-02T10:00:00Z,1000000,1,1,USD
1,txn002,StockIn,B_3_02-15-2049,2019-03-31T10:00:00Z,2019-04-02T10:00:00Z,1000000,1,1,USD
2,txn003,StockIn,C_3_02-15-2049,2019-03-31T10:00:00Z,2019-04-02T10:00:00Z,1000000,1,1,USD
3,txn004,StockIn,D_3_02-15-2049,2019-03-31T10:00:00Z,2019-04-02T10:00:00Z,1000000,1,1,USD


In [9]:
transaction_request = [
    lm.TransactionRequest(
        transaction_id=row["txn_id"],
        type=row["type"],
        instrument_identifiers={
            "Instrument/default/ClientInternal": row["client_id"]
        },
        transaction_date=row["trade_date"],
        settlement_date=row["settlement_date"],
        units=row["quantity"]*0.9,
        transaction_price=lm.TransactionPrice(price=row["price"], type="Price"),
        total_consideration=lm.CurrencyAndAmount(
            amount=row["total_consideration"], currency=row["currency"]
        )
    )
    for index, row in transactions.iterrows()
]


response = transaction_portfolios_api.upsert_transactions(
    scope=scope, code=portfolio_code, transaction_request=transaction_request
)

print(f"Transactions succesfully updated at time: {response.version.as_at_date}")

Transactions succesfully updated at time: 2023-02-21 18:38:09.562780+00:00


## 4. Add quotes for all defined bonds

We will now assign a quote prices to each of the bonds previously upserted into LUSID

In [10]:
bond_quotes = pd.read_csv("data/bond_quotes.csv")
bond_quotes.head(4)

,client_internal,date,price,currency
0,A_3_02-15-2049,15-07-2019,100,USD
1,A_3_02-15-2049,27-07-2019,100,USD
2,A_3_02-15-2049,06-08-2019,100,USD
3,B_3_02-15-2049,15-07-2019,100,USD


In [11]:
# Define a request body to upsert quotes to LUSID
instrument_quotes = {
    index: lm.UpsertQuoteRequest(
        quote_id=lm.QuoteId(
            quote_series_id=lm.QuoteSeriesId(
                provider="Lusid",
                instrument_id=row["client_internal"],
                instrument_id_type="ClientInternal",
                quote_type="Price",
                field="mid",
            ),
            effective_at=to_date(row["date"]).isoformat(),
        ),
        metric_value=lm.MetricValue(value=row["price"], unit=row["currency"]),
        scale_factor=100,
    )
    for index, row in bond_quotes.iterrows()
}
    
response = quotes_api.upsert_quotes(scope=scope, request_body=instrument_quotes)

if response.failed == {}:
    print(
        f"Quote successfully loaded into LUSID. {len(response.values)} quotes loaded."
    )

else:
    print(
        f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID."
    )

Quote successfully loaded into LUSID. 12 quotes loaded.


## 5. Create and run a valuation on the portfolio

Now that we have the bonds uploaded with corresponding quotes, we will create a valuation recipe.

### Create a valuation recipe

In [12]:
# Create a recipe to perform a valuation
configuration_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="bondValuationSimple",
    market=lm.MarketContext(
         market_rules=[
            lm.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            ),
            lm.MarketDataKeyRule(
                key="Quote.LusidInstrumentId.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                field="mid",
                quote_interval="5D.0D",
            )
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="ClientInternal",
            default_scope=scope,
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Bond",
                parameters="{}",
            )
        ]
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

### Create valuation function based on recipe

Once the recipe is defined, we can execute an evaluation on the bonds 

In [19]:
def get_val(date, portfolio_code):
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=scope, code="bondValuationSimple"),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Instrument/default/ClientInternal", "Value"),
            lm.AggregateSpec("Instrument/default/LusidInstrumentId", "Value"),
            lm.AggregateSpec("Holding/default/Units", "Value"),
            lm.AggregateSpec("Valuation/PV/Amount", "Value"),
            lm.AggregateSpec("Holding/default/Accrual", "Value"),
            lm.AggregateSpec("Valuation/PnL/Tm1", "Value")
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[lm.PortfolioEntityId(scope=scope, code=portfolio_code)],
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val_data = aggregration_api.get_valuation(valuation_request=valuation_request).data

    vals_df = pd.DataFrame(val_data)

    vals_df.rename(
        columns={
            "Instrument/default/Name": "InstrumentName",
            "Instrument/default/ClientInternal": "ClientInternal",
            "Valuation/PV/Amount": "Present Value",
            "Valuation/PnL/Tm1": "PnL (1-day)",
            "Holding/default/Accrual": "Accrued Interest",
        },
        inplace=True,
    )
    try:
        return vals_df.drop("Aggregation/Errors", axis=1)
    except:
        return vals_df

### Execute evaluation function

We will now run the evaluations. The accrued interest and portfolio value are shown. 

In [14]:
df_val = get_val("2019-07-27T10:00:00Z", portfolio_code)
df_val

,Present Value,InstrumentName,ClientInternal,Instrument/default/LusidInstrumentId,Holding/default/Units,Accrued Interest,PnL (1-day)
0,"901,617.19",C 3 02-15-2049,C_3_02-15-2049,LUID_00003D5O,"900,000.00","1,617.19",None
1,"901,617.19",D 3 02-15-2049,D_3_02-15-2049,LUID_00003D5P,"900,000.00","1,617.19",None
2,"901,669.35",B 3 02-15-2049,B_3_02-15-2049,LUID_00003D5N,"900,000.00","1,669.35",None
3,"901,784.48",A 3 02-15-2049,A_3_02-15-2049,LUID_00003D5M,"900,000.00","1,784.48",None


Finally, we will display the cash flows for the portfolio around EoM.

In [17]:
# Window set to EoM
window_start = datetime(2023,6,15)
window_end = datetime(2023,7,15)

# Date manipulation to add timezone
window_start = window_start.strftime('%Y-%m-%dT%H:%M:%SZ')
window_end = window_end.strftime('%Y-%m-%dT%H:%M:%SZ')

api_response = transaction_portfolios_api.get_portfolio_cash_flows(scope, portfolio_code, window_start=window_start, window_end=window_end)

# Join dataframes to show associated roll conventions
df = pd.DataFrame.from_dict(api_response.to_dict()['values'], orient='columns')
df['payment_date'] = df['payment_date'].dt.date
df = pd.merge(df, df_val, how='inner', left_on='source_instrument_id', right_on='Instrument/default/LusidInstrumentId')
df = df.merge(roll_df.to_frame(), how='outer', left_on='ClientInternal', right_on='id')
df = df.rename(columns={"InstrumentName": "instrument_name", "roll": "roll_convention", "ClientInternal": "client_internal"})


In [18]:
df[['instrument_name', 'roll_convention', 'payment_date', 'amount']]

,instrument_name,roll_convention,payment_date,amount
0,A 3 02-15-2049,Previous,2023-07-03,"2,250.00"
1,B 3 02-15-2049,NoAdjustment,2023-07-04,"2,250.00"
2,C 3 02-15-2049,ModifiedFollowing,2023-07-05,"2,250.00"
3,D 3 02-15-2049,Following,2023-07-05,"2,250.00"
